In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
device = "cuda" # the device to load the model onto

#model_path = "/home/imc/桌面/Qwen1.5-72B/pythonProject/model/LLM-Research/Meta-Llama-3-8B-Instruct"
#model_path = "/home/imc/桌面/aas_rag_generate/Qwen/Qwen2-7B"
model_path = "/home/imc/桌面/Qwen1.5-72B/pythonProject/model/qwen/Qwen1___5-14B-Chat"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards: 100%|██████████| 8/8 [00:25<00:00,  3.21s/it]


In [17]:
prompt = '''
你好，请问你是谁？
'''
messages = [
    {"role": "system", "content": ""},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print(text)



<|im_start|>system
<|im_end|>
<|im_start|>user

你好，请问你是谁？
<|im_end|>
<|im_start|>assistant



In [1]:
# text = '''
# franka机器人程序依赖的库如下：
# from omni.isaac.franka import Franka
# from omni.isaac.core import World
# from omni.isaac.core.objects import DynamicCuboid
# from omni.isaac.franka.controllers import PickPlaceController
# import numpy as np
# jetbot机器人依赖的库如下：
# import carb
# import numpy as np
# from omni.isaac.core import World
# from omni.isaac.nucleus import get_assets_root_path
# from omni.isaac.wheeled_robots.controllers.differential_controller import DifferentialController
# from omni.isaac.wheeled_robots.robots import WheeledRobot
# ur10机器人依赖的库如下：
# import numpy as np
# from omni.isaac.universal_robots import UR10
# from omni.isaac.core import World
# from omni.isaac.universal_robots.controllers.pick_place_controller import PickPlaceController
# from omni.isaac.core.objects import DynamicCuboid
# 请你将这些库依赖整合，输出python的import代码段
# '''

In [33]:
# Prepare your prompts
prompt = "告诉我大模型相关的一些介绍."
messages = [
    {"role": "system", "content": "你是一个python编程高手."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
告诉我大模型相关的一些介绍.<|im_end|>
<|im_start|>assistant



In [38]:
text = '''
以下例程为将ur10机器人放置在（0,0,0）处并初始化的代码段：
ur10 = my_world.scene.add(UR10(prim_path="/World/UR10", name="my_UR10", gripper_usd=None, attach_gripper=True, position=[0,0,0]))
ur10.set_joints_default_state(
    positions=np.array([-np.pi / 2, -np.pi / 2, -np.pi / 2, -np.pi / 2, np.pi / 2, 0])
)
ur10.gripper.set_default_state(opened=True)
my_controller = PickPlaceController(name="pick_place_controller", gripper=ur10.gripper, robot_articulation=ur10)
articulation_controller = ur10.get_articulation_controller()
请你仿照其写出将ur10机械臂A放置在（10,10,10）处，并初始化的代码，变量名使用ur10_a，另一个ur10机械臂B,使用变量名ur10_b，放在（-10,0,0）并初始化的代码
'''


In [40]:
model_inputs = tokenizer([text], return_tensors="pt").to(device)
input_ids = tokenizer.encode(text,return_tensors='pt')
attention_mask = torch.ones(input_ids.shape,dtype=torch.long,device=device)
generated_ids = model.generate(
    model_inputs.input_ids,
    attention_mask=attention_mask,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

```python
# 创建第一个UR10机械臂A
ur10_a = my_world.scene.add(UR10(prim_path="/World/UR10", name="ur10_a", gripper_usd=None, attach_gripper=True, position=[10, 10, 10]))

# 设置机械臂A的关节初始位置
ur10_a.set_joints_default_state(positions=np.array([-np.pi / 2, -np.pi / 2, -np.pi / 2, -np.pi / 2, np.pi / 2, 0]))

# 打开机械臂A的夹爪
ur10_a.gripper.set_default_state(opened=True)

# 创建第二个UR10机械臂B
ur10_b = my_world.scene.add(UR10(prim_path="/World/UR10", name="ur10_b", gripper_usd=None, attach_gripper=True, position=[-10, 0, 0]))

# 设置机械臂B的关节初始位置
ur10_b.set_joints_default_state(positions=np.array([-np.pi / 2, -np.pi / 2, -np.pi / 2, -np.pi / 2, np.pi / 2, 0]))

# 打开机械臂B的夹爪
ur10_b.gripper.set_default_state(opened=True)

# 创建控制器
controller_a = PickPlaceController(name="controller_a", gripper=ur10_a.gripper, robot_articulation=ur10_a)
controller_b = PickPlaceController(name="controller_b", gripper=ur10_b.gripper, robot_articulation=ur10_b)
articulation_controller_a = ur10_a.get_articulation_controller()
articulation_c